In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim
import torchvision
from torchvision import datasets, transforms

class ShuffleUnit(nn.Module):
    def __init__(self, in_channels, out_channels, groups=3, stride=1):
        super(ShuffleUnit, self).__init__()

        self.groups = groups
        self.stride = stride
        mid_channels = out_channels // 4

        # 第1个逐点卷积层
        self.conv1 = nn.Conv2d(in_channels, mid_channels, kernel_size=1, groups=1, bias=False)
        self.bn1 = nn.BatchNorm2d(mid_channels)
        self.relu = nn.ReLU(inplace=True)

        # 通道洗牌操作
        self.channel_shuffle = ChannelShuffle(groups)

        # 第2个深度可分离卷积层
        self.conv2 = nn.Conv2d(mid_channels, mid_channels, kernel_size=3, stride=stride, padding=1, groups=mid_channels, bias=False)
        self.bn2 = nn.BatchNorm2d(mid_channels)

        # 第3个逐点卷积层
        self.conv3 = nn.Conv2d(mid_channels, out_channels, kernel_size=1, groups=1, bias=False)
        self.bn3 = nn.BatchNorm2d(out_channels)

    def forward(self, x):
        out = self.conv1(x)
        out = self.bn1(out)
        out = self.relu(out)

        out = self.channel_shuffle(out)

        out = self.conv2(out)
        out = self.bn2(out)

        out = self.conv3(out)
        out = self.bn3(out)

        return out

class ChannelShuffle(nn.Module):
    def __init__(self, groups):
        super(ChannelShuffle, self).__init__()
        self.groups = groups

    def forward(self, x):
        batch_size, num_channels, height, width = x.data.size()
        channels_per_group = num_channels // self.groups

        # reshape
        x = x.view(batch_size, self.groups, channels_per_group, height, width)

        # transpose
        x = torch.transpose(x, 1, 2).contiguous()

        # reshape back
        x = x.view(batch_size, -1, height, width)

        return x

class ShuffleNet(nn.Module):
    def __init__(self, groups=3, num_classes=10):
        super(ShuffleNet, self).__init__()

        self.conv1 = nn.Conv2d(3, 24, kernel_size=3, stride=1, padding=1, bias=False)
        self.bn1 = nn.BatchNorm2d(24)
        self.relu = nn.ReLU(inplace=True)

        self.stage2 = self._make_stage(24, 144, 4, groups)
        self.stage3 = self._make_stage(144, 288, 8, groups)
        self.stage4 = self._make_stage(288, 576, 4, groups)

        self.avg_pool = nn.AdaptiveAvgPool2d(1)
        self.fc = nn.Linear(576, num_classes)

    def _make_stage(self, in_channels, out_channels, num_blocks, groups):
        stage = []
        for i in range(num_blocks):
            if i == 0:
                stage.append(ShuffleUnit(in_channels, out_channels, groups=groups, stride=2))
            else:
                stage.append(ShuffleUnit(out_channels, out_channels, groups=groups, stride=1))
        return nn.Sequential(*stage)

    def forward(self, x):
        x = self.conv1(x)
        x = self.bn1(x)
        x = self.relu(x)

        x = self.stage2(x)
        x = self.stage3(x)
        x = self.stage4(x)

        x = self.avg_pool(x)
        x = x.view(x.size(0), -1)
        x = self.fc(x)

        return x

# 设置一些超参数
num_classes = 10
learning_rate = 0.001
batch_size = 64

# 加载数据集，这里使用CIFAR-10作为示例
transform = transforms.Compose([
    transforms.RandomHorizontalFlip(),
    transforms.ToTensor(),
    transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5)),
])

train_dataset = datasets.CIFAR10(root='./data', train=True, download=True, transform=transform)
train_loader = torch.utils.data.DataLoader(train_dataset, batch_size=batch_size, shuffle=True)

# 初始化模型、损失函数和优化器
model = ShuffleNet(groups=3, num_classes=num_classes)
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=learning_rate)

# 训练模型
num_epochs = 5
for epoch in range(num_epochs):
    for inputs, labels in train_loader:
        optimizer.zero_grad()
        outputs = model(inputs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

    print(f'Epoch [{epoch+1}/{num_epochs}], Loss: {loss.item():.4f}')

# 保存模型
torch.save(model.state_dict(), 'shufflenet_model.pth')
